In [0]:
# Step 1: Create widgets for dynamic values
dbutils.widgets.text("storage_accounts", "sadbw5", "Storage Account Names")
dbutils.widgets.text("container_names", "container1", "Container Names")
dbutils.widgets.text("credential_names", "dbrcredentials1", "Credential Names")
dbutils.widgets.text("location_names", "extlocation1", "External Location Names")
dbutils.widgets.text("access_connector_ids", "/subscriptions/11939925-8a3f-47a2-b68c-a6038b46bd4b/resourceGroups/pragati/providers/Microsoft.Databricks/accessConnectors/testconnector", "Access Connector ID")
#dbutils.widgets.text("catalog_names", "mycatalog1", "Comma-separated Catalog Names")
dbutils.widgets.text("division_codes", "prm", "Division Codes")
dbutils.widgets.text("application_names", "dbx360", "Application Names")
dbutils.widgets.text("environments", "dev", "Environment")

In [0]:
# Step 2: Retrieve widget values
storage_accounts = dbutils.widgets.get("storage_accounts").split(",")  # Convert to a list
container_names = dbutils.widgets.get("container_names").split(",")  # Convert to a list
credential_names = dbutils.widgets.get("credential_names").split(",")  # Convert to a list
location_names = dbutils.widgets.get("location_names").split(",")  # Convert to a list
managed_identity_ids = dbutils.widgets.get("access_connector_ids").split(",")  # Convert to a list of Managed Identity IDs
#catalog_names = dbutils.widgets.get("catalog_names").split(",")  # Convert to a list
division_codes = dbutils.widgets.get("division_codes").split(",")
application_names = dbutils.widgets.get("application_names").split(",")
environments = dbutils.widgets.get("environments").split(",")

In [0]:
# print("Storage Accounts Length:", len(storage_accounts))
# print("Container Names Length:", len(container_names))
# print("Credential Names Length:", len(credential_names))
# print("Location Names Length:", len(location_names))
# print("Catalog Names Length:", len(catalog_names))
# print("Managed Identity IDs Length:", len(managed_identity_ids))

In [0]:
# Step 3: Validate if all lists have the same length
# if not (len(storage_accounts) == len(container_names) == len(credential_names) == len(location_names) == len(catalog_names) == len(managed_identity_ids)):
#     raise ValueError("Mismatch in the number of storage accounts, containers, credentials, external locations, catalogs, and managed identities. Ensure they match.")
if not (len(storage_accounts) == len(container_names) == len(credential_names) == len(location_names) == len(managed_identity_ids)) == len(division_codes) == len(application_names) == len(environments):
    raise ValueError("Mismatch in the number of storage accounts, containers, credentials, external locations and managed identities. Ensure they match.")

In [0]:
# Step 4: Generate Catalog Names based on the combination of division, application, and environment
catalog_names = []

for division_code, application_name, environment in zip(division_codes, application_names, environments):

    catalog_name = f"{division_code.upper()}-{application_name}-{environment}"
    catalog_names.append(catalog_name)

# Print out the constructed catalog names to verify
print("Constructed Catalog Names:", catalog_names)

In [0]:
# Step 5: Get Databricks instance and authentication token dynamically
DATABRICKS_INSTANCE = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
# Get authentication headers dynamically (No need for manual token)
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
import requests
import json

In [0]:
# Step 6: Function to create storage credentials using Managed Identity
def create_storage_credential(credential_name, managed_identity_id):
    # API Endpoint for creating storage credentials
    url = f"{DATABRICKS_INSTANCE}/api/2.1/unity-catalog/storage-credentials"
    
    # JSON payload for creating storage credentials
    payload = {
        "name": credential_name,
        "azure_managed_identity": {
            "access_connector_id": managed_identity_id
        },
        "comment": f"Storage credential for {credential_name} using managed identity"
    }

    # Send request to create storage credential
    response = requests.post(url, headers=headers, json=payload)

    # Handle response
    if response.status_code == 200:
        print(f"Storage credential '{credential_name}' created successfully.")
    else:
        print(f"Failed to create storage credential '{credential_name}': {response.text}")
		

In [0]:
# Step 7: Loop through each set of values and create storage credentials, external locations, and Unity catalogs
for storage_account, container_name, credential_name, location_name, catalog_name, managed_identity_id in zip(storage_accounts, container_names, credential_names, location_names, catalog_names, managed_identity_ids):

    # Create storage credentials dynamically
    create_storage_credential(credential_name, managed_identity_id)

    # External Location URL
    abfss_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/"

    # Create External Location
    query_create_location = f"""
    CREATE EXTERNAL LOCATION {location_name} 
    URL '{abfss_path}'
    WITH (STORAGE CREDENTIAL {credential_name})
    COMMENT 'External data source for {container_name} in storage account {storage_account}';
    """
    
    # Create Unity Catalog
    query_create_catalog = f"""
    CREATE CATALOG IF NOT EXISTS {catalog_name}
      MANAGED LOCATION '{abfss_path}';
    """

    # Execute SQL queries for creating External Location and Catalog
    print(f"Executing Query: Create External Location - {location_name}")
    display(spark.sql(query_create_location))

    print(f"Executing Query: Create Catalog - {catalog_name}")
    display(spark.sql(query_create_catalog))

In [0]:
# Step 8: Create Bronze, Silver, and Gold schemas dynamically
schemas = ["bronze", "silver", "gold"]

for schema in schemas:
    for catalog_name, container_name in zip(catalog_names, container_names):
        query_create_schema = f"""
        CREATE SCHEMA IF NOT EXISTS `{catalog_name}`.{schema}
        MANAGED LOCATION 'abfss://{container_name}@{storage_accounts[0]}.dfs.core.windows.net/{schema}/';
        """
        
        print(f"Executing Query: Create Schema - {catalog_name}.{schema}")
        display(spark.sql(query_create_schema))



In [0]:
# Step 9: Create Table in the Bronze schema
query_use_schema = f"USE `{catalog_names[0]}`.bronze;"
query_create_table = f"""
CREATE TABLE IF NOT EXISTS `{catalog_names[0]}`.bronze.test_table (
    id INT,
    name STRING
) LOCATION 'abfss://{container_names[0]}@{storage_accounts[0]}.dfs.core.windows.net/bronze/test_table/';
"""

print(f"Executing Query: USE Schema - {catalog_names[0]}.bronze")
display(spark.sql(query_use_schema))

print("Executing Query: Create Table - test_table in Bronze Schema")
display(spark.sql(query_create_table))


In [0]:
%sql
CREATE GROUP test WITH USER `abc@gmail.com`;

In [0]:
%sql
-- Add the user to the group
ALTER GROUP test ADD USER `xyz@gmail.com`;

In [0]:
%sql
GRANT USE CATALOG ON CATALOG mycatalog1 TO test;

In [0]:
%sql
GRANT USE CATALOG,USE SCHEMA ON CATALOG mycatalog TO test;

In [0]:
%sql
REVOKE USE SCHEMA ON CATALOG mycatalog FROM test;

In [0]:
%sql
--Read operations:
GRANT BROWSE, EXECUTE, READ VOLUME ON CATALOG mycatalog TO test1;

In [0]:
%sql
--Read and Edit operations:
GRANT MODIFY, REFRESH, WRITE VOLUME ON CATALOG mycatalog TO test2;

In [0]:
dbutils.widgets.removeAll()